In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

vgsales = pd.read_csv("vgsales.csv")
vgsales = pd.DataFrame(vgsales)

vgsales['Year'] = vgsales['Year'].fillna(-1)
vgsales['Year'] = vgsales['Year'].astype(int)
vgsales.set_index('Rank', inplace=True, drop=True)
       


In [79]:
vgsales_year = vgsales.groupby(["Year"],as_index=False)[["Global_Sales"]].sum()
vgsales_year

#for year in vgsales_year["Year"]:
#    for sales in vgsales_year["Global_Sales"]:
        
#    print(year)

100.07999999999994
11.379999999999999
35.77000000000001
28.859999999999996
16.790000000000003
50.360000000000014
53.940000000000005
37.07
21.739999999999995
47.22


# Teste de lógica da operação de cálculo de vendas totais por ano:

In [52]:
teste = [[2000, 23.7],
         [2000, 20.1],
         [2001, 34],
         [2001, 47],
         [2001, 78],
         [2002, 88]]

teste2 = {}
sales = 0

for lista in teste:
    sales += lista[1]
    teste2[lista[0]] = sales


teste2

{2000: 43.8, 2001: 202.8, 2002: 290.8}